### Markov chains

A machine is either working or in repair center. If it is working today, then there is a 95% chance that it will be working tomorrow. If it is in repair center today, then theree is a 40% chance that it will be working tomorrow.

In [15]:
import numpy as np

P = np.matrix([[.95, .05], [.4, .6]])
print(P)

[[ 0.95  0.05]
 [ 0.4   0.6 ]]


Stationary distribution

$$\pi={\pi}P$$

In [17]:
# Searching for stationary distribution.

S, U = np.linalg.eig(P.T)  # eigenvalues and eigenvector
stationary = np.array(U[:, np.where(np.abs(S - 1.) < 1e-8)[0][0]].flat) # Search vector corresponding to eigenvalue of 1
stationary = stationary / np.sum(stationary)  # normalize
print(stationary)

[ 0.88888889  0.11111111]


Thus machine is broken 1 out of every 9 days in average.